In [2]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime

Cargo los dataframes originales

In [3]:
df_original = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)

Me quedo con las columnas de interes

In [4]:
df_reducido = df_original.loc[:,['person','event','sku','model','city','region','country']]
df_reducido.head()

,person,event,sku,model,city,region,country
0,4886f805,viewed product,9288.0,Samsung Galaxy J7 Prime,NaN,NaN,NaN
1,ad93850f,viewed product,304.0,iPhone 5s,NaN,NaN,NaN
2,0297fc1e,viewed product,6888.0,iPhone 6S,NaN,NaN,NaN
3,2d681dd8,viewed product,11890.0,iPhone 7,NaN,NaN,NaN
4,cccea85e,viewed product,7517.0,LG G4 H818P,NaN,NaN,NaN


Construyo un dataframe que tiene las personas, con las regiones a las que pertenecen, y solo las que realizaron viewed product

In [5]:
df_region_person = df_original.loc[df_original['country'].notnull(),['person','region','country']]

In [6]:
df_region_person['counter'] = 1
df_prueba = df_region_person.groupby(['person','region','country'])['counter'].sum().reset_index()

In [7]:
df_region_person= df_prueba.sort_values(by='counter',ascending = False).drop_duplicates(subset='person',keep='first')

In [8]:
df_region_person = df_region_person.rename(columns={'region':'most_visited_region'})

In [9]:
df_region_person = df_region_person.drop(['counter'],axis=1)

In [10]:
df_reducido = df_reducido.merge(df_region_person, left_on='person', right_on='person', how='left')
df_reducido.head()

,person,event,sku,model,city,region,country_x,most_visited_region,country_y
0,4886f805,viewed product,9288.0,Samsung Galaxy J7 Prime,NaN,NaN,NaN,Rio de Janeiro,Brazil
1,ad93850f,viewed product,304.0,iPhone 5s,NaN,NaN,NaN,Sao Paulo,Brazil
2,0297fc1e,viewed product,6888.0,iPhone 6S,NaN,NaN,NaN,Rio de Janeiro,Brazil
3,2d681dd8,viewed product,11890.0,iPhone 7,NaN,NaN,NaN,Sao Paulo,Brazil
4,cccea85e,viewed product,7517.0,LG G4 H818P,NaN,NaN,NaN,Sao Paulo,Brazil


In [11]:
df_reducido = df_reducido.loc[df_reducido['sku'].notnull(),['person','event','sku','model','most_visited_region','country_y']]
df_reducido.head()

,person,event,sku,model,most_visited_region,country_y
0,4886f805,viewed product,9288.0,Samsung Galaxy J7 Prime,Rio de Janeiro,Brazil
1,ad93850f,viewed product,304.0,iPhone 5s,Sao Paulo,Brazil
2,0297fc1e,viewed product,6888.0,iPhone 6S,Rio de Janeiro,Brazil
3,2d681dd8,viewed product,11890.0,iPhone 7,Sao Paulo,Brazil
4,cccea85e,viewed product,7517.0,LG G4 H818P,Sao Paulo,Brazil


In [12]:
df_reducido.columns = ['person','event','sku','model','most_visited_region','pais']

Ahora tengo un dataframe que tiene por persona, la region a la que pertenece y los modelos con los que interactuo. Ahora, solo quiero los que miro:

In [13]:
df_only_vp = df_reducido.loc[df_reducido['event'] == 'viewed product', :]

Ahora quiero, por persona, el producto que mas visito:

In [14]:
df_most_visited_per_person = df_only_vp.groupby(['person','model']).size().reset_index()\
                                                                            .rename(columns={0:'Cantidad_visitas'})

In [15]:
df_most_visited_per_person = df_most_visited_per_person.sort_values(by='Cantidad_visitas',ascending=False)\
                                                        .drop_duplicates(subset='person',keep='first')

In [16]:
df_most_visited_per_person = df_most_visited_per_person.merge(df_region_person, on='person',how='left')

In [17]:
df_most_visited_per_person.head()

,person,model,Cantidad_visitas,most_visited_region,country
0,b1f4dbf6,iPhone 5s,832,Sao Paulo,Brazil
1,171e75cb,iPhone 6S,679,Espirito Santo,Brazil
2,6f19cfd9,iPhone 6S Plus,622,Bahia,Brazil
3,6abd2bf1,iPhone 6,603,Sao Paulo,Brazil
4,9bf968c5,iPhone 6,558,Bahia,Brazil


Ahora, un dataframe que tiene el top 5 de cada region de los productos mas visitados.

In [18]:
df_most_visited_per_region = df_original.loc[df_original['event'] == 'viewed product', ['person','region', 'model', 'sku']]
df_most_visited_per_region = df_most_visited_per_region.drop(columns='region')
df_most_visited_per_region = df_most_visited_per_region.merge(df_region_person, on='person', how='left')
df_most_visited_per_region = df_most_visited_per_region.rename(columns={'most_visited_region':'region'})
df_most_visited_per_region.head()

,person,model,sku,region,country
0,4886f805,Samsung Galaxy J7 Prime,9288.0,Rio de Janeiro,Brazil
1,ad93850f,iPhone 5s,304.0,Sao Paulo,Brazil
2,0297fc1e,iPhone 6S,6888.0,Rio de Janeiro,Brazil
3,2d681dd8,iPhone 7,11890.0,Sao Paulo,Brazil
4,cccea85e,LG G4 H818P,7517.0,Sao Paulo,Brazil


Aca calculo la cantidad de visitas por producto y por regions

In [19]:
df_most_visited_per_region = df_most_visited_per_region.groupby(['region','model']).size().reset_index().rename(columns={0:'Visitas'})

In [20]:
df_most_visited_per_region.sort_values(by='Visitas',ascending=False).head()

,region,model,Visitas
4379,Sao Paulo,iPhone 6,30029
4381,Sao Paulo,iPhone 6S,27944
4378,Sao Paulo,iPhone 5s,26365
4383,Sao Paulo,iPhone 7,18624
4971,Unknown,iPhone 6,15331


Ahora, solo conservamos el top 5 de cada una de las regiones

In [21]:
df_most_visited_per_region = df_most_visited_per_region.sort_values(by='Visitas',ascending=False)\
                                                       .groupby('region').head(5)

Construyo un diccionario que tiene como clave la region, y como valor una lista con los 5 productos mas visitados por los usuarios de esa region (algunas listas tienen menos de 5 items por la falta de datos de algunas regiones)

In [22]:
df_top_5_por_region = df_most_visited_per_region.groupby('region')
top_5_region = {}
regiones_del_df = df_most_visited_per_region['region'].drop_duplicates().dropna()
for region in regiones_del_df:
    top_5_region[region] = list(df_top_5_por_region.get_group(region)['model'])

In [23]:
top_5_region['Sao Paulo']

['iPhone 6', 'iPhone 6S', 'iPhone 5s', 'iPhone 7', 'Samsung Galaxy S7 Edge']

Ahora, utilizando ese diccionario y el dataframe del producto mas visitado por cada persona, construyo una columna que decide si la persona visitó mas veces un producto que entra en el top 5 de su region.

In [24]:
df_most_visited_per_person = df_most_visited_per_person.dropna()
df_most_visited_per_person['person'].describe()


count        36931
unique       36931
top       03bfba07
freq             1
Name: person, dtype: object

In [25]:
df_most_visited_per_person['busco_top_5_visitas'] = [s.model in top_5_region[s.most_visited_region] for s in df_most_visited_per_person.itertuples()]

In [26]:
df_most_visited_per_person['busco_top_5_visitas'] = df_most_visited_per_person['busco_top_5_visitas'].astype(int)

In [27]:
df_most_visited_per_person.shape

(36931, 6)

In [28]:
df_features = df_original['person'].drop_duplicates().to_frame()
df_features = df_features.merge(df_most_visited_per_person, on='person', how='left')

In [29]:
df_features.head()

,person,model,Cantidad_visitas,most_visited_region,country,busco_top_5_visitas
0,4886f805,Samsung Galaxy J7 Prime,4.0,Rio de Janeiro,Brazil,0.0
1,ad93850f,iPhone 5s,17.0,Sao Paulo,Brazil,1.0
2,0297fc1e,iPhone 6,213.0,Rio de Janeiro,Brazil,1.0
3,2d681dd8,iPhone 7,6.0,Sao Paulo,Brazil,1.0
4,cccea85e,Motorola Moto G4 Plus,252.0,Sao Paulo,Brazil,0.0


Voy a hacer un df que tiene la cantidad de visitas de una misma persona, en su region, por modelo que haya visitado

In [30]:
df_cantidad_visitas = df_original.loc[df_original['event'] == 'viewed product', ['person','model','event']]
df_cantidad_visitas = df_cantidad_visitas.merge(df_region_person, on='person', how='left')
df_cantidad_visitas = df_cantidad_visitas.groupby(['most_visited_region','person','model']).size().reset_index()
df_cantidad_visitas.loc[df_cantidad_visitas['most_visited_region'] == 'Sao Paulo',:].sort_values(by=0,ascending=False).head()

,most_visited_region,person,model,0
163170,Sao Paulo,b1f4dbf6,iPhone 5s,832
146149,Sao Paulo,6abd2bf1,iPhone 6,603
163175,Sao Paulo,b1f4dbf6,iPhone SE,554
146151,Sao Paulo,6abd2bf1,iPhone 6S,514
156792,Sao Paulo,97b0c0d1,iPhone SE,477


Y ahora, con este df calculo cuanto es el promedio de visitas de un modelo por region

In [31]:
df_promedio_de_visitas = df_cantidad_visitas.groupby(['most_visited_region','model'])[0].mean().reset_index()
df_promedio_de_visitas.loc[df_promedio_de_visitas['most_visited_region'] == 'Sao Paulo',:].sort_values(by=0,ascending=False).head()

,most_visited_region,model,0
4340,Sao Paulo,Samsung Galaxy Young 2 Duos TV,11.000000
4381,Sao Paulo,iPhone 6S,10.442451
4378,Sao Paulo,iPhone 5s,9.904207
4379,Sao Paulo,iPhone 6,9.832678
4236,Sao Paulo,Motorola Moto G4 Plus,9.100159


In [32]:
df_promedio_de_visitas = df_promedio_de_visitas.rename(columns={0:'Promedio_visitas_producto'})

Mergeo esa informacion en el dataframe final

In [33]:
df_features = df_features.merge(df_promedio_de_visitas, on=['most_visited_region','model'], how='left')
df_features.head()

,person,model,Cantidad_visitas,most_visited_region,country,busco_top_5_visitas,Promedio_visitas_producto
0,4886f805,Samsung Galaxy J7 Prime,4.0,Rio de Janeiro,Brazil,0.0,5.043919
1,ad93850f,iPhone 5s,17.0,Sao Paulo,Brazil,1.0,9.904207
2,0297fc1e,iPhone 6,213.0,Rio de Janeiro,Brazil,1.0,9.325721
3,2d681dd8,iPhone 7,6.0,Sao Paulo,Brazil,1.0,8.860133
4,cccea85e,Motorola Moto G4 Plus,252.0,Sao Paulo,Brazil,0.0,9.100159


Nuevo feature: ver si la persona visito el producto mas veces que el promedio

In [34]:
df_features['visito_mas_que_el_promedio'] = df_features['Cantidad_visitas'] > df_features['Promedio_visitas_producto']
df_features['visito_mas_que_el_promedio'].value_counts()

False    21159
True     17670
Name: visito_mas_que_el_promedio, dtype: int64

Ahora, saco cuanto tiempo gasto una persona en la pagina visitando productos

In [35]:
df_tiempo_por_persona = df_original.loc[df_original['event'] == 'viewed product',['person','timestamp']]

In [36]:
df_tiempo_por_persona['dia'] = pd.to_datetime(df_tiempo_por_persona['timestamp']).dt.day
df_tiempo_por_persona['mes'] = pd.to_datetime(df_tiempo_por_persona['timestamp']).dt.month
df_tiempo_por_persona.head(2)

,person,timestamp,dia,mes
0,4886f805,2018-05-18 00:11:59,18,5
1,ad93850f,2018-05-18 00:11:27,18,5


In [37]:
df_tiempo_por_persona['DeltaTime'] = pd.to_datetime(df_tiempo_por_persona['timestamp'])
df_tiempo_por_persona['time'] = df_tiempo_por_persona['DeltaTime'].dt.time

def make_delta(entry):
    h, m, s = str(entry).split(':')
    return datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s))

df_tiempo_por_persona['DeltaTime'] = df_tiempo_por_persona['time'].apply(lambda entry: make_delta(entry))

In [38]:
df_tiempo_por_persona.head(5)

,person,timestamp,dia,mes,DeltaTime,time
0,4886f805,2018-05-18 00:11:59,18,5,00:11:59,00:11:59
1,ad93850f,2018-05-18 00:11:27,18,5,00:11:27,00:11:27
2,0297fc1e,2018-05-18 00:11:16,18,5,00:11:16,00:11:16
3,2d681dd8,2018-05-18 00:11:14,18,5,00:11:14,00:11:14
4,cccea85e,2018-05-18 00:11:09,18,5,00:11:09,00:11:09


In [39]:
df_horas_vp = df_tiempo_por_persona.groupby(['person','mes','dia']).agg({'DeltaTime': ['max','min']}).reset_index()
df_horas_vp['hora_por_dia'] = df_horas_vp['DeltaTime']['max'] - df_horas_vp['DeltaTime']['min']
df_horas_vp.head()

person mes dia DeltaTime          hora_por_dia
                          max      min             
0  00091926   5   3  22:09:59 22:08:35     00:01:24
1  00091926   5   5  23:45:04 03:43:24     20:01:40
2  00091926   5   6  02:05:38 02:04:55     00:00:43
3  00091926   5   7  00:37:20 00:31:13     00:06:07
4  00091926   5   8  23:42:50 23:30:44     00:12:06

In [40]:
df_horas_vp = df_horas_vp.groupby('person')['hora_por_dia'].sum().reset_index()
df_horas_vp.head()

,person,hora_por_dia
0,00091926,2 days 16:48:21
1,00091a7a,0 days 00:01:00
2,000ba417,0 days 04:23:53
3,000c79fe,0 days 00:07:09
4,000e4d9e,0 days 17:07:22


In [41]:
def segundos_totales(serie):
    return serie.total_seconds()


df_horas_vp['horas_totales'] = df_horas_vp['hora_por_dia'].apply(segundos_totales)
df_horas_vp.head()

,person,hora_por_dia,horas_totales
0,00091926,2 days 16:48:21,233301.0
1,00091a7a,0 days 00:01:00,60.0
2,000ba417,0 days 04:23:53,15833.0
3,000c79fe,0 days 00:07:09,429.0
4,000e4d9e,0 days 17:07:22,61642.0


In [42]:
df_horas_vp['horas_totales'] = df_horas_vp['horas_totales'] / 3600
df_horas_vp.head()

,person,hora_por_dia,horas_totales
0,00091926,2 days 16:48:21,64.805833
1,00091a7a,0 days 00:01:00,0.016667
2,000ba417,0 days 04:23:53,4.398056
3,000c79fe,0 days 00:07:09,0.119167
4,000e4d9e,0 days 17:07:22,17.122778


In [43]:
df_horas_vp = df_horas_vp.drop(columns=['hora_por_dia'])
df_horas_vp = df_horas_vp.rename(columns={'horas_totales':'horas_mirando_productos'})

In [44]:
df_horas_vp.head()

,person,horas_mirando_productos
0,00091926,64.805833
1,00091a7a,0.016667
2,000ba417,4.398056
3,000c79fe,0.119167
4,000e4d9e,17.122778


In [45]:
df_features = df_features.merge(df_horas_vp, on='person', how='left')

In [46]:
df_features.head()

,person,model,Cantidad_visitas,most_visited_region,country,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos
0,4886f805,Samsung Galaxy J7 Prime,4.0,Rio de Janeiro,Brazil,0.0,5.043919,False,0.315278
1,ad93850f,iPhone 5s,17.0,Sao Paulo,Brazil,1.0,9.904207,True,22.058056
2,0297fc1e,iPhone 6,213.0,Rio de Janeiro,Brazil,1.0,9.325721,True,191.342500
3,2d681dd8,iPhone 7,6.0,Sao Paulo,Brazil,1.0,8.860133,False,0.092222
4,cccea85e,Motorola Moto G4 Plus,252.0,Sao Paulo,Brazil,0.0,9.100159,True,50.758611


Lo mergeo al dataframe final

In [47]:
df_features = df_features.rename(columns={'most_visited_region':'region_persona', 'country':'pais'})
df_features.head()

,person,model,Cantidad_visitas,region_persona,pais,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos
0,4886f805,Samsung Galaxy J7 Prime,4.0,Rio de Janeiro,Brazil,0.0,5.043919,False,0.315278
1,ad93850f,iPhone 5s,17.0,Sao Paulo,Brazil,1.0,9.904207,True,22.058056
2,0297fc1e,iPhone 6,213.0,Rio de Janeiro,Brazil,1.0,9.325721,True,191.342500
3,2d681dd8,iPhone 7,6.0,Sao Paulo,Brazil,1.0,8.860133,False,0.092222
4,cccea85e,Motorola Moto G4 Plus,252.0,Sao Paulo,Brazil,0.0,9.100159,True,50.758611


Otro feature: ver si una persona aumento la cantidad de registros que realizó mes a mes

In [48]:
df_actividad_mes = df_original.loc[:,['person','timestamp','event']]
df_actividad_mes['datetime'] = pd.to_datetime(df_actividad_mes['timestamp'])
df_actividad_mes['mes'] = df_actividad_mes['datetime'].dt.month

In [49]:
df_registros_por_mes = df_actividad_mes.groupby(['person','mes']).size().reset_index()
df_registros_por_mes.groupby('person').get_group('00091926')

,person,mes,0
1,00091926,5,448


In [50]:
df_personas = df_registros_por_mes[['person']].drop_duplicates() #Las personas del dataframe inicial

In [51]:
for i in range(1,6):
    df_personas['mes'] = i
    df_personas = df_personas.merge(df_registros_por_mes, on=['person','mes'], how='left')

df_personas = df_personas.drop(columns=['mes'])
df_personas.columns = ['person','mes1','mes2','mes3','mes4','mes5']
df_personas.head()

,person,mes1,mes2,mes3,mes4,mes5
0,0008ed71,NaN,NaN,NaN,NaN,6.0
1,00091926,NaN,NaN,NaN,NaN,448.0
2,00091a7a,NaN,NaN,10.0,NaN,NaN
3,000ba417,NaN,NaN,NaN,NaN,206.0
4,000c79fe,NaN,NaN,NaN,NaN,17.0


In [52]:
df_personas = df_personas.fillna(0)
df_personas_registros_mes = df_personas
df_personas_registros_mes.head()

,person,mes1,mes2,mes3,mes4,mes5
0,0008ed71,0.0,0.0,0.0,0.0,6.0
1,00091926,0.0,0.0,0.0,0.0,448.0
2,00091a7a,0.0,0.0,10.0,0.0,0.0
3,000ba417,0.0,0.0,0.0,0.0,206.0
4,000c79fe,0.0,0.0,0.0,0.0,17.0


In [53]:
df_personas_registros_mes['siempre_incrementando'] = (df_personas_registros_mes['mes1'] < df_personas_registros_mes['mes2']) &\
                                                     (df_personas_registros_mes['mes2'] < df_personas_registros_mes['mes3']) &\
                                                     (df_personas_registros_mes['mes3'] < df_personas_registros_mes['mes4']) &\
                                                     (df_personas_registros_mes['mes4'] < df_personas_registros_mes['mes5'])

In [54]:
df_personas_registros_mes['inc_ultimo_mes'] = (df_personas_registros_mes['mes4'] < df_personas_registros_mes['mes5'])
df_personas_registros_mes['inc_ultimos_dos_mes'] = (df_personas_registros_mes['mes4'] < df_personas_registros_mes['mes5']) &\
                                                   ((df_personas_registros_mes['mes3'] < df_personas_registros_mes['mes4']))

In [55]:
df_personas_registros_mes['promedio_ingreso_mensual'] = (df_personas_registros_mes['mes1'] +\
                                                         df_personas_registros_mes['mes2'] +\
                                                         df_personas_registros_mes['mes3'] +\
                                                         df_personas_registros_mes['mes4'] +\
                                                         df_personas_registros_mes['mes5']) / 5

In [56]:
df_features.head()

,person,model,Cantidad_visitas,region_persona,pais,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos
0,4886f805,Samsung Galaxy J7 Prime,4.0,Rio de Janeiro,Brazil,0.0,5.043919,False,0.315278
1,ad93850f,iPhone 5s,17.0,Sao Paulo,Brazil,1.0,9.904207,True,22.058056
2,0297fc1e,iPhone 6,213.0,Rio de Janeiro,Brazil,1.0,9.325721,True,191.342500
3,2d681dd8,iPhone 7,6.0,Sao Paulo,Brazil,1.0,8.860133,False,0.092222
4,cccea85e,Motorola Moto G4 Plus,252.0,Sao Paulo,Brazil,0.0,9.100159,True,50.758611


In [57]:
df_personas_registros_mes.head()

,person,mes1,mes2,mes3,mes4,mes5,siempre_incrementando,inc_ultimo_mes,inc_ultimos_dos_mes,promedio_ingreso_mensual
0,0008ed71,0.0,0.0,0.0,0.0,6.0,False,True,False,1.2
1,00091926,0.0,0.0,0.0,0.0,448.0,False,True,False,89.6
2,00091a7a,0.0,0.0,10.0,0.0,0.0,False,False,False,2.0
3,000ba417,0.0,0.0,0.0,0.0,206.0,False,True,False,41.2
4,000c79fe,0.0,0.0,0.0,0.0,17.0,False,True,False,3.4


In [58]:
df_features = df_features.merge(df_personas_registros_mes, on='person', how='left')

In [59]:
df_features.head()

,person,model,Cantidad_visitas,region_persona,pais,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos,mes1,mes2,mes3,mes4,mes5,siempre_incrementando,inc_ultimo_mes,inc_ultimos_dos_mes,promedio_ingreso_mensual
0,4886f805,Samsung Galaxy J7 Prime,4.0,Rio de Janeiro,Brazil,0.0,5.043919,False,0.315278,0.0,0.0,0.0,0.0,9.0,False,True,False,1.8
1,ad93850f,iPhone 5s,17.0,Sao Paulo,Brazil,1.0,9.904207,True,22.058056,0.0,0.0,0.0,0.0,65.0,False,True,False,13.0
2,0297fc1e,iPhone 6,213.0,Rio de Janeiro,Brazil,1.0,9.325721,True,191.342500,120.0,48.0,87.0,139.0,173.0,False,True,True,113.4
3,2d681dd8,iPhone 7,6.0,Sao Paulo,Brazil,1.0,8.860133,False,0.092222,0.0,0.0,0.0,0.0,26.0,False,True,False,5.2
4,cccea85e,Motorola Moto G4 Plus,252.0,Sao Paulo,Brazil,0.0,9.100159,True,50.758611,0.0,0.0,0.0,0.0,836.0,False,True,False,167.2


In [60]:
df_features = df_features.fillna(0)
df_features['siempre_incrementando'] = df_features['siempre_incrementando'].astype(int)
df_features['inc_ultimo_mes'] = df_features['inc_ultimo_mes'].astype(int)
df_features['inc_ultimos_dos_mes'] = df_features['inc_ultimos_dos_mes'].astype(int)

El mas buscado por usuario en lo ultimos meses

In [61]:
df_mas_buscado_mensual = df_original.loc[df_original['event'] == 'viewed product', ['person','timestamp','model'] ]

In [62]:
df_mas_buscado_mensual['dt'] = pd.to_datetime(df_mas_buscado_mensual['timestamp'])
df_mas_buscado_mensual['mes'] = df_mas_buscado_mensual['dt'].dt.month

In [63]:
df_persona_mes_modelo = df_mas_buscado_mensual.groupby(['person','mes','model']).size().reset_index()

In [64]:
df_persona_mes_modelo = df_persona_mes_modelo.sort_values(by=0,ascending=False)
df_persona_mes_modelo = df_persona_mes_modelo.rename(columns={0:'amount'})

In [65]:
df_persona_mes_modelo = df_persona_mes_modelo.groupby(['person','mes']).head(1).reset_index()

In [66]:
df_personas = df_mas_buscado_mensual[['person']].drop_duplicates() #Las personas del dataframe inicial

In [67]:
df_persona_mes_modelo = df_persona_mes_modelo.drop(columns=['index','amount'])

In [68]:
df_persona_mes_modelo.head()

,person,mes,model
0,aa297476,5,iPhone 6S
1,b793fc69,5,iPhone SE
2,b1f4dbf6,3,iPhone 5s
3,069c5926,5,iPhone 6S
4,b1f4dbf6,4,iPhone 5s


In [69]:
for i in range(1,6):
    df_personas['mes'] = i
    df_personas = df_personas.merge(df_persona_mes_modelo, on=['person','mes'], how='left')

df_personas = df_personas.drop(columns=['mes'])
df_personas.columns = ['person','mes1','mes2','mes3','mes4','mes5']
df_personas.head()

,person,mes1,mes2,mes3,mes4,mes5
0,4886f805,NaN,NaN,NaN,NaN,Samsung Galaxy J7 Prime
1,ad93850f,NaN,NaN,NaN,NaN,iPhone 5s
2,0297fc1e,iPhone 6,iPhone 6,iPhone 6,iPhone 6S,iPhone 6S
3,2d681dd8,NaN,NaN,NaN,NaN,iPhone 7
4,cccea85e,NaN,NaN,NaN,NaN,Motorola Moto G4 Plus


In [70]:
df_interes_por_mes = df_personas
df_interes_por_mes['mismo_interes_ultimos_dos_meses'] = np.where(df_interes_por_mes['mes4'] == df_interes_por_mes['mes5'],
                                                                True, False)

In [71]:
df_interes_por_mes = df_interes_por_mes.drop(columns=['mes1','mes2','mes3','mes4','mes5'])

In [72]:
df_features = df_features.merge(df_interes_por_mes, on='person', how='outer')

In [73]:
df_features = df_features.fillna(0)

In [74]:
df_features['mismo_interes_ultimos_dos_meses'] = df_features['mismo_interes_ultimos_dos_meses'].astype(int)

In [75]:
df_features.head()

,person,model,Cantidad_visitas,region_persona,pais,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos,mes1,mes2,mes3,mes4,mes5,siempre_incrementando,inc_ultimo_mes,inc_ultimos_dos_mes,promedio_ingreso_mensual,mismo_interes_ultimos_dos_meses
0,4886f805,Samsung Galaxy J7 Prime,4.0,Rio de Janeiro,Brazil,0.0,5.043919,False,0.315278,0.0,0.0,0.0,0.0,9.0,0,1,0,1.8,0
1,ad93850f,iPhone 5s,17.0,Sao Paulo,Brazil,1.0,9.904207,True,22.058056,0.0,0.0,0.0,0.0,65.0,0,1,0,13.0,0
2,0297fc1e,iPhone 6,213.0,Rio de Janeiro,Brazil,1.0,9.325721,True,191.342500,120.0,48.0,87.0,139.0,173.0,0,1,1,113.4,1
3,2d681dd8,iPhone 7,6.0,Sao Paulo,Brazil,1.0,8.860133,False,0.092222,0.0,0.0,0.0,0.0,26.0,0,1,0,5.2,0
4,cccea85e,Motorola Moto G4 Plus,252.0,Sao Paulo,Brazil,0.0,9.100159,True,50.758611,0.0,0.0,0.0,0.0,836.0,0,1,0,167.2,0


Tipo de evento que mas genero en el ultimo mes

In [76]:
df_eventos_personas = df_original.loc[:,['person','event','timestamp']]
df_eventos_personas['datetime'] = pd.to_datetime(df_eventos_personas['timestamp'])
df_eventos_personas['mes'] = df_eventos_personas['datetime'].dt.month

In [77]:
df_eventos_personas_mes_5 = df_eventos_personas.groupby('mes').get_group(5)

In [78]:
df_mayor_evento_mes_5 = df_eventos_personas_mes_5.groupby(['person','event']).size().reset_index()

In [79]:
df_mayor_evento_mes_5 = df_mayor_evento_mes_5\
                        .sort_values(by=0,ascending = False)\
                        .drop_duplicates(subset='person',keep='first')

In [80]:
df_mayor_evento_mes_5.person.describe()

count        37143
unique       37143
top       03bfba07
freq             1
Name: person, dtype: object

In [81]:
df_mayor_evento_mes_5 = df_mayor_evento_mes_5.drop(columns=0)

In [82]:
df_mayor_evento_mes_5.columns = ['person','evento_predominante_mes_5']

In [83]:
df_mayor_evento_mes_5.shape

(37143, 2)

In [84]:
df_features = df_features.merge(df_mayor_evento_mes_5, on='person', how='outer')

In [85]:
df_features.head()

,person,model,Cantidad_visitas,region_persona,pais,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos,mes1,mes2,mes3,mes4,mes5,siempre_incrementando,inc_ultimo_mes,inc_ultimos_dos_mes,promedio_ingreso_mensual,mismo_interes_ultimos_dos_meses,evento_predominante_mes_5
0,4886f805,Samsung Galaxy J7 Prime,4.0,Rio de Janeiro,Brazil,0.0,5.043919,False,0.315278,0.0,0.0,0.0,0.0,9.0,0,1,0,1.8,0,viewed product
1,ad93850f,iPhone 5s,17.0,Sao Paulo,Brazil,1.0,9.904207,True,22.058056,0.0,0.0,0.0,0.0,65.0,0,1,0,13.0,0,viewed product
2,0297fc1e,iPhone 6,213.0,Rio de Janeiro,Brazil,1.0,9.325721,True,191.342500,120.0,48.0,87.0,139.0,173.0,0,1,1,113.4,1,viewed product
3,2d681dd8,iPhone 7,6.0,Sao Paulo,Brazil,1.0,8.860133,False,0.092222,0.0,0.0,0.0,0.0,26.0,0,1,0,5.2,0,viewed product
4,cccea85e,Motorola Moto G4 Plus,252.0,Sao Paulo,Brazil,0.0,9.100159,True,50.758611,0.0,0.0,0.0,0.0,836.0,0,1,0,167.2,0,viewed product


## Division de features segun su tipo

In [86]:
df_features_numericos = df_features[['person','Cantidad_visitas',\
                                     'Promedio_visitas_producto',\
                                     'horas_mirando_productos',\
                                    'mes1',\
                                    'mes2',\
                                    'mes3',\
                                    'mes4',\
                                    'mes5',\
                                    'promedio_ingreso_mensual']]
df_features_numericos.columns = ['person','Cantidad_visitas',\
                                     'Promedio_visitas_producto',\
                                     'horas_mirando_productos',\
                                    'visitas_mes1',\
                                    'visitas_mes2',\
                                    'visitas_mes3',\
                                    'visitas_mes4',\
                                    'visitas_mes5',\
                                    'promedio_ingreso_mensual']

In [87]:
df_features_bool = df_features[['person', 'busco_top_5_visitas',\
                               'visito_mas_que_el_promedio',\
                               'siempre_incrementando',\
                               'inc_ultimo_mes',\
                               'mismo_interes_ultimos_dos_meses']]

In [88]:
df_features_categoricos = df_features[['person','model', 'region_persona','pais','evento_predominante_mes_5']]
df_features_categoricos.columns = ['person','modelo_mas_visitado','region_persona','pais','evento_predominante_mes_5']
df_features_categoricos.head()

,person,modelo_mas_visitado,region_persona,pais,evento_predominante_mes_5
0,4886f805,Samsung Galaxy J7 Prime,Rio de Janeiro,Brazil,viewed product
1,ad93850f,iPhone 5s,Sao Paulo,Brazil,viewed product
2,0297fc1e,iPhone 6,Rio de Janeiro,Brazil,viewed product
3,2d681dd8,iPhone 7,Sao Paulo,Brazil,viewed product
4,cccea85e,Motorola Moto G4 Plus,Sao Paulo,Brazil,viewed product


In [89]:
df_features = df_features_categoricos.merge(df_features_bool, on='person',how='outer')

In [90]:
df_features = df_features.merge(df_features_numericos ,on='person', how='outer')

In [91]:
df_original.drop_duplicates(subset= 'person').shape

(38829, 23)

In [92]:
df_features['person'].drop_duplicates().shape

(38829,)

In [93]:
df_features_numericos.to_csv(path_or_buf = 'features_numericos.csv', index = False)
df_features_bool.to_csv(path_or_buf = 'features_bool.csv', index = False)
df_features_categoricos.to_csv(path_or_buf = 'features_categoricos.csv', index = False)